<a href="https://colab.research.google.com/github/DynamicLLM/LLM2024/blob/main/mathSolverMulVerification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Description:
This program is an interactive math-solving tool that uses OpenAI's GPT to solve and verify mathematical problems. The program guides the user through the problem-solving process step by step and provides a verification mechanism. Users can select from suggested verification methods, such as graphical solutions, numerical solutions, substitution verification, or mathematical theorems, to confirm the accuracy of the solution. This makes the tool highly adaptable to various types of mathematical problems.

Instructions to Use:
1. Run the program and input a math question when prompted (e.g., "how can I solve 8x + 7 = -23").
2. Review the list of suggested verification methods and select one or more by entering their corresponding numbers. If no method is selected, the program will default to "Substitution verification."
3. The program will call GPT to solve the question and display the solution steps along with the final answer.
4. It will then verify the solution using the chosen methods and output the verification result.
5. Review the verification result to confirm the solution's correctness.


In [8]:
from pydantic import BaseModel
from openai import OpenAI

class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

# Initialize OpenAI client
client = OpenAI(api_key="sbvhCp4tqdcwA")


In [23]:

def solve_math_question():
    def suggest_verification_method(question):
        return [
            "Graphical solution",
            "Numerical solution",
            "Substitution verification",
            "Mathematical theorems"
        ]

    # Step 1: Ask the user to input a math question
    math_question = input("Enter a math question (e.g., how can I solve 8x + 7 = -23): ")

    # Step 2: Ask the user to input a verifying method
    suggested_methods = suggest_verification_method(math_question)
    print("Suggested Verification Methods:")
    for i, method in enumerate(suggested_methods, 1):
        print(f"{i}. {method}")

    selected_methods = input("Select verification methods by entering the corresponding numbers (e.g., 1, 3): ")
    selected_methods_indices = [int(num.strip()) - 1 for num in selected_methods.split(",") if num.strip().isdigit()]
    verifying_methods = [suggested_methods[i] for i in selected_methods_indices if 0 <= i < len(suggested_methods)]

    if not verifying_methods:
        verifying_methods = ["Substitution verification"]  # Default method if none selected

    verifying_method = ", ".join(verifying_methods)

    # Step 3: Solve the math question using OpenAI
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
            {"role": "user", "content": math_question},
        ],
        response_format=MathReasoning,
    )

    # Parse the response
    math_reasoning = completion.choices[0].message.parsed

    # Step 4: Output the math reasoning steps
    print("\nMath Reasoning Steps:")
    for step in math_reasoning.steps:
        print(f"Explanation: {step.explanation}")
        print(f"Output: {step.output}\n")

    # Step 5: Output the final answer
    final_answer = math_reasoning.final_answer
    print(f"Final Answer: {final_answer}\n")

    # Step 6: Verify the result using GPT
    print("Now let's verify the solution.")
    verification_prompt = (
        f"Verify the solution to the question '{math_question}' using {verifying_method}, but without repeating solving the problem again. "
        f"The final answer provided was {final_answer}. "
        "Confirm whether the solution is correct."
    )
    print("verification_prompt", verification_prompt)
    verification_completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a math verification assistant. Confirm if the solution is correct."},
            {"role": "user", "content": verification_prompt},
        ]
    )

    verification_response = verification_completion.choices[0].message.content
    print("Verification Result:")
    print(verification_response)

# Run the program interactively
if __name__ == "__main__":
    solve_math_question()


Enter a math question (e.g., how can I solve 8x + 7 = -23): 8+x=0
Suggested Verification Methods:
1. Graphical solution
2. Numerical solution
3. Substitution verification
4. Mathematical theorems
Select verification methods by entering the corresponding numbers (e.g., 1, 3): 1,4

Math Reasoning Steps:
Explanation: To isolate the variable x, we need to get rid of 8 from the left side of the equation. We can do this by subtracting 8 from both sides.
Output: x = 0 - 8

Explanation: Now we simplify the right side of the equation. 0 - 8 is -8.
Output: x = -8

Final Answer: x = -8

Now let's verify the solution.
verification_prompt Verify the solution to the question '8+x=0' using Graphical solution, Mathematical theorems, but withour repeating solving the problem again. The final answer provided was x = -8. Confirm whether the solution is correct.
Verification Result:
To verify the solution of the equation \(8 + x = 0\) graphically and mathematically without re-solving, we can proceed as fo